# SageMaker Experiments

This notebook shows how you can use SageMaker Experiment Management Python SDK to organize, track, compare, and evaluate your machine learning (ML) model training experiments.

You can track artifacts for experiments, including data sets, algorithms, hyper-parameters, and metrics. Experiments executed on SageMaker such as SageMaker Autopilot jobs and training jobs will be automatically tracked. You can also track artifacts for additional steps within an ML workflow that come before/after model training e.g. data pre-processing or post-training model evaluation.

The APIs also let you search and browse your current and past experiments, compare experiments, and identify best performing models.

### Setup 

Uncomment (remove the #) the command below to install sagemaker-experiments

In [ ]:
#!pip install sagemaker-experiments

### Imports 

In [1]:
import pandas as pd
import numpy as np
import boto3
import time

from sagemaker.analytics import ExperimentAnalytics
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
from sagemaker.session import Session
import sagemaker

from smexperiments.trial_component import TrialComponent
from smexperiments.experiment import Experiment
from smexperiments.tracker import Tracker
from smexperiments.trial import Trial

### Essentials

In [2]:
session = sagemaker.Session()
role = get_execution_role()

In [3]:
boto_session = boto3.Session()
sagemaker_boto_client = boto_session.client('sagemaker')

### Upload Data to S3

In [4]:
WORK_DIRECTORY = './DATA'

train_data_s3 = session.upload_data(f'{WORK_DIRECTORY}/train', key_prefix='sklearn-clf/train')
test_data_s3 = session.upload_data(f'{WORK_DIRECTORY}/test', key_prefix='sklearn-clf/test')

### Create a Processing Tracker 

In [5]:
with Tracker.create(display_name='Preprocessing', sagemaker_boto_client=sagemaker_boto_client) as processing_tracker:
    processing_tracker.log_parameters({
        'scaling-strategy': 'min-max',
    })
    # we can log the s3 uri to the dataset we just uploaded
    processing_tracker.log_input(name='training-data', 
                      media_type='s3/uri', 
                      value=train_data_s3)

### Create an Experiment

Create an experiment to track all the model training iterations. Experiments are a great way to organize your data science work. You can create experiments to organize all your model development work for : [1] a business use case you are addressing (e.g. create experiment named “customer churn prediction”), or [2] a data science team that owns the experiment (e.g. create experiment named “marketing analytics experiment”), or [3] a specific data science and ML project. Think of it as a “folder” for organizing your “files”.

In [6]:
experiment = Experiment.create(
    experiment_name=f'Sklearn-Classifier-{int(time.time())}', 
    description='Sklearn Classifier Example', 
    sagemaker_boto_client=sagemaker_boto_client)

In [7]:
experiment

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f20c25b69e8>,experiment_name='Sklearn-Classifier-1604596195',description='Sklearn Classifier Example',tags=None,experiment_arn='arn:aws:sagemaker:us-east-1:892313895307:experiment/sklearn-classifier-1604596195',response_metadata={'RequestId': '2a208ffb-a431-4553-bcb6-aaf7e47e8a72', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2a208ffb-a431-4553-bcb6-aaf7e47e8a72', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Thu, 05 Nov 2020 17:09:55 GMT'}, 'RetryAttempts': 0})

### Create a Trial

Now create a Trial for each training run to track the it's inputs, parameters, and metrics.<br>
We will also create a TrialComponent from the <b>Processing Tracker</b> we created before, and add to the Trial.<br>
This will enrich the Trial with the parameters we captured from the data processing stage.<br>
Note the execution of the following code takes a while.

In [8]:
candidates = [3, 5, 10] # number of neighbors

In [9]:
processing_trial_component = processing_tracker.trial_component

In [10]:
for i, n_neighbors in enumerate([3, 7, 9]):
    trial_name = f'sklearn-clf-{n_neighbors}-{int(time.time())}'
    trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=experiment.experiment_name,
        sagemaker_boto_client=sagemaker_boto_client,
    )
    # Associate the Processing trial component with the current trial
    trial.add_trial_component(processing_trial_component)
    
    estimator = SKLearn(entry_point='train.py',
                    train_instance_type='ml.m5.large',
                    train_instance_count=1,
                    framework_version='0.23-1',
                    role=role,
                    hyperparameters= {'n-neighbors': n_neighbors},
                    metric_definitions=[
                            {'Name':'train:loss', 'Regex':'Train Loss: (.*?);'},
                            {'Name':'test:loss', 'Regex':'Test Average loss: (.*?),'},
                            {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*?)%;'}
                    ],
                    enable_sagemaker_metrics=True # IMPORTANT
                )
                        
    training_job_name = 'training-job-{}'.format(int(time.time()))
    
    # Associate the estimator with an Experiment and a Trial
    estimator.fit(
        inputs={'training': train_data_s3}, 
        job_name=training_job_name,
        experiment_config={
            'TrialName': trial.trial_name,
            'TrialComponentDisplayName': 'Training',
        },
        wait=True, 
    )
    time.sleep(2)

FileNotFoundError: [Errno 2] No such file or directory: 'train.py'